In [1]:
from itertools import combinations,permutations
import pandas as pd
import re

In [2]:
transactions = {'T1' : 'abcdef',
                'T2' : 'abcg',
                'T3' : 'abdh',
                'T4' : 'bcdek',
                'T5' : 'abc'}

### Initialisation

#### sorting each transaction

In [3]:
for i in transactions.keys():
    transactions[i] = ''.join(sorted(transactions[i]))

#### check if subset

In [4]:
def checkSubset(sset,set):
    if len(sset) > len(set):
        return False
    newSet = []
    for num,item in enumerate(list(permutations(set))):
        newSet.append(''.join(list(item)))
    for itemset in newSet:
        if sset in itemset:
            return True
    return False

#### calculate support function

In [5]:
def calculateSupport(itemset, transactions):
    support_itemset = 0
    for i in transactions.values():
        if checkSubset(itemset,i):
            support_itemset += 1
    
    return support_itemset

#### generate candidate set


In [6]:
candidateSet = {}
for i in transactions.values():
    for j in i:
        if j not in candidateSet.keys():
            candidateSet[j] = calculateSupport(j,transactions)


#### input the minimum support

In [7]:
minSup = int(input("Enter the minimum support : "))

#### generate frequent itemset function

In [8]:
#pruning the candidate set for frequent itemset

freqSet = {}
def generateFreqSet(candidateSet):
    for i in candidateSet.keys():
        if candidateSet[i] >= minSup:
            freqSet[i] = candidateSet[i]
    return freqSet
generateFreqSet(candidateSet)

freqSet

{'a': 4, 'b': 5, 'c': 4, 'd': 3, 'e': 2}

#### initialisation of number of iterations(k)

In [9]:
k = 2

### Algorithm


#### generating candidate sets from frequent itemsets


In [10]:
def removeDuplicates(word):
    word = ''.join(sorted(list(set(word))))
    return word

#### generate candidate set function

In [11]:
def generateCandidateSet(freqSet,K):
    new_keys = list(combinations(freqSet.keys(),K))
    for num,i in enumerate(new_keys):
        new_keys[num] = list(set(i))
        res = removeDuplicates(''.join(new_keys[num])) #remove all duplicates while maintaining order
        new_keys[num] = res
    candidateSet = {}
    for i in new_keys:
        i = ''.join(sorted(i))
        candidateSet[i] = calculateSupport(i,transactions)
    return candidateSet

In [31]:
def printConfidence(freqSet):
    for item in freqSet.keys():
        for item_range in range(len(freqSet)):
            for i in list(combinations(item, item_range)):
                i = ''.join(list(i))
                if len(i) != 0:
                    print(f"{i} -> {''.join(list(set(item)-set(i)))} : ",calculateSupport(item,transactions)/calculateSupport(i,transactions))


#### Pruning frequent itemsets

In [12]:
def pruneFreqSet(freqSet):
    rm_arr = []
    for i in freqSet.keys():
        newSet_keys = list(freqSet.keys())
        newSet_keys.remove(i) #removing itself as an item so that it does not count itself as a subset
        for j in newSet_keys:
            if checkSubset(i,j):
                rm_arr.append(i)
    for i in rm_arr:
        freqSet.pop(i)
        
    return freqSet

#### Algorithm

In [32]:
while len(candidateSet.keys()) > 0:    
    candidateSet = generateCandidateSet(freqSet,k)
    #removing subsets present in freqSet
    for cand_item in candidateSet.keys():
        rm_keys = []
        for freq_item in freqSet.keys():
            if re.search(f'[{freq_item}]',cand_item):
                rm_keys.append(freq_item)
            list(set(rm_keys)) #creating unique values in the keys to be removed array
        for keys in rm_keys:
            freqSet.pop(keys)
    freqSet = generateFreqSet(candidateSet)
    freqSet = pruneFreqSet(freqSet)
    k += 1

print("Frequent Itemsets are : ", list(freqSet.keys()))
print("Association Rules are : ")
printConfidence(freqSet)

Frequent Itemsets are :  ['abc', 'abd', 'bcde']
Association Rules are : 
a -> bc :  0.75
b -> ca :  0.6
c -> ba :  0.75
ab -> c :  0.75
ac -> b :  1.0
bc -> a :  0.75
a -> bd :  0.5
b -> ad :  0.4
d -> ba :  0.6666666666666666
ab -> d :  0.5
ad -> b :  1.0
bd -> a :  0.6666666666666666
b -> ced :  0.4
c -> bed :  0.5
d -> bce :  0.6666666666666666
e -> bcd :  1.0
bc -> ed :  0.5
bd -> ce :  0.6666666666666666
be -> cd :  1.0
cd -> be :  1.0
ce -> bd :  1.0
de -> bc :  1.0
